In [1]:
import torch
import random
from seeker.random import RandomSelectPairSeeker, RandomSelectSeeker, RangeGenSeeker, DistributionGenSeeker
from seeker.gradiant_based import WhiteboxSeeker, BlackboxSeeker, FoolSeeker
from utils import UnfairMetric, load_model
from data import adult, german, loans_default
from train_dnn import get_data
from models.model import MLP
from distances.normalized_mahalanobis_distances import ProtectedSEDistances
from distances.sensitive_subspace_distances import LogisticRegSensitiveSubspace
from distances.binary_distances import BinaryDistance
from IPython.display import display

%load_ext autoreload
%autoreload 2

In [2]:
# data
data_name = 'adult'
path = 'new' if data_name in ['census', 'loans_default'] else 'no_norm'
use_sensitive_attr = True
sensitive_vars = ['sex_Male']
# model
rand_seed = 0
trainer_name = 'std'
rho=0.1
_note = ''
note=_note if trainer_name == 'std' else _note + f'rho={rho}'
# others
device = 'cpu'

In [3]:
data_choices = {
    'adult': adult,
    'german': german,
    'loans_default': loans_default
}
data = data_choices[data_name]
data_gen = data.Generator(use_sensitive_attr, sensitive_vars, device)

dataset, train_dl, test_dl = get_data(data, rand_seed, sensitive_vars=sensitive_vars)
dataset.use_sensitive_attr = use_sensitive_attr
in_dim = dataset.dim_feature()
out_dim = 2

all_X, all_y = dataset.get_all_data(), dataset.labels

In [4]:
model = MLP(in_dim, out_dim, data_gen=data_gen, n_layers=4, norm=False)
load_model(model, data_name, trainer_name, use_sensitive_attr=use_sensitive_attr, \
           sensitive_vars=sensitive_vars, id=rand_seed, note=note, path=path)

In [5]:
distance_x_Causal = ProtectedSEDistances()
distance_x_LR = LogisticRegSensitiveSubspace()
distance_y = BinaryDistance()

if use_sensitive_attr:
    distance_x_Causal.fit(num_dims=dataset.dim_feature(), data_gen=data_gen, sensitive_idx=dataset.sensitive_idxs)
    chosen_dx = distance_x_Causal
else:
    sensitive_ = dataset.data[:, dataset.sensitive_idxs]
    distance_x_LR.fit(dataset.get_all_data(), data_gen=data_gen, data_SensitiveAttrs=sensitive_)
    chosen_dx = distance_x_LR

In [6]:
x = torch.zeros(chosen_dx.num_dims)
pert = 10*torch.diag(torch.ones_like(x))
g = torch.zeros_like(x)
for i in range(g.shape[0]):
    g[i] = chosen_dx(x, x+pert[i])
epsilon = (1/torch.min(g[g!=0])).item()
epsilon

99997992.0

In [7]:
epsilon = 1e8
unfair_metric = UnfairMetric(dx=chosen_dx, dy=distance_y, epsilon=epsilon)

In [8]:
def show_result(result):
    pair, n_query = result[0], result[1]
    if len(result) == 3:
        print(f'n_iters = {result[2]}')
    if pair != None:
        display(data_gen.feature_dataframe(data = pair), n_query)
    else:
        display('not found')

In [9]:
# random.seed(422)
# torch.manual_seed(422)

# select_seeker = RandomSelectSeeker(model=model, unfair_metric=unfair_metric, data=dataset.get_all_data(), data_gen=data_gen)
# for _ in range(3):
#     show_result(select_seeker.seek(max_query=1e5))

In [10]:
# random.seed(422)
# torch.manual_seed(422)

# distribution_seeker = DistributionGenSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen)
# for _ in range(3):
#     show_result(distribution_seeker.seek(max_query=1e5))

In [11]:
# random.seed(422)
# torch.manual_seed(422)

# range_seeker = RangeGenSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen)
# for _ in range(3):
#     show_result(range_seeker.seek(max_query=5e4, p_stop=0.7))

In [12]:
# random.seed(42)
# torch.manual_seed(42)

# test_seeker = WhiteboxSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen)
# for i in range(3):
#     display(f'try: {i}')
#     show_result(test_seeker.seek(origin_lr=0.01, max_query=1e5, lamb=1))

In [13]:
random.seed(422)
torch.manual_seed(422)
import numpy as np  

for xi in [0.0001, 0.001, 0.01, 0.1]:
    print(xi)
    test_seeker = BlackboxSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen, easy=False)
    n = []
    for i in range(10):
        display(f'try: {i}')
        result = test_seeker.seek(origin_lr=0.1, max_query=5e4, lamb=0.5)
        # show_result(result)
        n.append(result[1])
    print(np.mean(n))
    print(np.std(n))

0.0001


'try: 0'

/home/liuyuanhao/miniconda3/envs/torch1.9/lib/python3.9/site-packages/torch/_functorch/deprecated.py:58: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


'try: 1'

'try: 2'

...

'try: 6'

'try: 7'

'try: 8'

'try: 9'

3046.7
5553.495620777961


In [14]:
# random.seed(422)
# torch.manual_seed(422)

# test_seeker = BlackboxSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen, easy=True)
# for i in range(3):
#     display(f'try: {i}')
#     show_result(test_seeker.seek(origin_lr=0.1, max_query=1e5, lamb=1))